## 파일 불러오기 

- 국내 코로나 확진자-지역별 일일 현황_2021년 11월 5주차.xlsx

In [24]:
import pandas as pd
import openpyxl
import xlsxwriter

s = '국내 코로나 확진자-지역별 일일 현황_2021년 11월 5주차.xlsx'

In [25]:
city_list = ['강원','경기','경남','경북','광주','대구','대전','부산','서울','세종','울산','인천','전남','전북','제주','충남','충북','합계']

## 저장할 파일 생성

In [26]:
# 현재 날짜에 맞춰 '2021년  N월 M주차_주요 국가별 코로나 확진자-백신접종자 현황'으로 저장하기
import datetime as dt
from datetime import datetime, timedelta

def get_date(y, m, d):
    s = f'{y:04d}-{m:02d}-{d:02d}'
    return datetime.strptime(s, '%Y-%m-%d')

def get_week_no(y, m, d):
    target = get_date(y, m, d)
    firstday = target.replace(day = 1)
    if firstday.weekday() == 6:
        origin = firstday
    elif firstday.weekday() < 3:
        origin = firstday - timedelta(days=firstday.weekday() + 1)
    else:
        origin = firstday + timedelta(days = 6-firstday.weekday())
    return (target - origin).days // 7 + 1

In [27]:
n = dt.datetime.now()
writer = pd.ExcelWriter('국내 코로나 확진자-지역별 월별 통계_{}년 {}월.xlsx'
                        .format(n.year, n.month, get_week_no(n.year, n.month-1, n.day)), 
                        engine = 'xlsxwriter')

## 도시별 월별 확진자 구한 후 시트별로 저장하기

In [28]:
for city in city_list:
    # 파일 불러오기 
    df = pd.read_excel(s, engine = 'openpyxl', sheet_name=city)
    
    print(city)
    
    # 확진자 수 차분하기
    # 공공데이터포털의 일일값의 합과 누적값에 차이 있어 명확한 가이드라인이 주어지지 않으면 누적값을 차분에 계산
    # 확진자를 일일확진자로 변경
    # 전날 (누적)확진자 수 - 당일 (누적)확진자 수 = 당일 확진자 수
    df['일일 확진자 수'] = df['확진자 수'].diff()
    
    # na 값을 0으로 대체
    df = df.fillna(0)

    # float -> int
    df = df.astype({'일일 확진자 수': int})
    
    # 날짜 월까지만 남기기
    # 특정문자 기준으로 문자열 나누고 새로운 열 추가
    df['년'] = df.날짜.str.split(' ').str[0]
    df['월'] = df.날짜.str.split(' ').str[1]
    
    # 날짜를 지우고 년 월만 맨 앞으로
    new_df = df[['년', '월', '시도명','일일 확진자 수']]
    # 일일 확진자 수 -> 월별 확진자 수
    new_df = new_df.rename(columns={'일일 확진자 수':'월별 확진자 수'})

    # 날짜로 groupby
    # 년, 월 기준으로 sum()
    result_df = new_df.groupby(['년', '월', '시도명'], as_index=False).sum()
    print(result_df)
    
    # 시트별로 저장하기 
    result_df.to_excel(writer, index = False, sheet_name=city)
    
writer.save()

강원
        년    월 시도명  월별 확진자 수
0   2020년  01월  강원         0
1   2020년  02월  강원         7
2   2020년  03월  강원        29
3   2020년  04월  강원        17
4   2020년  05월  강원         4
5   2020년  06월  강원         8
6   2020년  07월  강원         9
7   2020년  08월  강원       115
8   2020년  09월  강원        35
9   2020년  10월  강원        55
10  2020년  11월  강원       373
11  2020년  12월  강원       555
12  2021년  01월  강원       488
13  2021년  02월  강원       169
14  2021년  03월  강원       466
15  2021년  04월  강원       354
16  2021년  05월  강원       501
17  2021년  06월  강원       374
18  2021년  07월  강원       934
19  2021년  08월  강원       967
20  2021년  09월  강원       977
21  2021년  10월  강원       762
22  2021년  11월  강원      1441
경기
        년    월 시도명  월별 확진자 수
0   2020년  01월  경기         2
1   2020년  02월  경기        74
2   2020년  03월  경기       400
3   2020년  04월  경기       200
4   2020년  05월  경기       164
5   2020년  06월  경기       367
6   2020년  07월  경기       339
7   2020년  08월  경기      1714
8   2020년  09월  경기      1123
9   2020

전남
        년    월 시도명  월별 확진자 수
0   2020년  01월  전남         0
1   2020년  02월  전남         2
2   2020년  03월  전남         7
3   2020년  04월  전남         6
4   2020년  05월  전남         4
5   2020년  06월  전남         5
6   2020년  07월  전남        14
7   2020년  08월  전남       104
8   2020년  09월  전남        27
9   2020년  10월  전남        16
10  2020년  11월  전남       239
11  2020년  12월  전남       136
12  2021년  01월  전남       189
13  2021년  02월  전남       110
14  2021년  03월  전남        54
15  2021년  04월  전남       140
16  2021년  05월  전남       410
17  2021년  06월  전남       185
18  2021년  07월  전남       387
19  2021년  08월  전남       565
20  2021년  09월  전남       484
21  2021년  10월  전남       459
22  2021년  11월  전남       876
전북
        년    월 시도명  월별 확진자 수
0   2020년  01월  전북         1
1   2020년  02월  전북         4
2   2020년  03월  전북         8
3   2020년  04월  전북         5
4   2020년  05월  전북         3
5   2020년  06월  전북         6
6   2020년  07월  전북        12
7   2020년  08월  전북        48
8   2020년  09월  전북        41
9   2020

## 하나 미리 해보기

In [6]:
df = pd.read_excel(s, engine = 'openpyxl', sheet_name=city_list[0])
df

,시도명,날짜,확진자 수,지역발생 수
0,강원,2020년 01월 20일 00시,0,0
1,강원,2020년 01월 21일 00시,0,0
2,강원,2020년 01월 22일 00시,0,0
3,강원,2020년 01월 23일 00시,0,0
4,강원,2020년 01월 24일 00시,0,0
...,...,...,...,...
675,강원,2021년 11월 26일 00시,8344,53
676,강원,2021년 11월 27일 00시,8437,93
677,강원,2021년 11월 28일 00시,8500,61
678,강원,2021년 11월 29일 00시,8556,55


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 680 entries, 0 to 679
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시도명     680 non-null    object
 1   날짜      680 non-null    object
 2   확진자 수   680 non-null    int64 
 3   지역발생 수  680 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 21.4+ KB


In [8]:
# 공공데이터포털의 일일값의 합과 누적값에 차이 있어
# 명확한 가이드라인이 주어지지 않으면 누적값을 차분에 계산

# 누적확진자를 일일확진자로 변경
# 전날 (누적)확진자 수 - 당일 (누적)확진자 수 = 당일 확진자 수
df['일일 확진자 수'] = df['확진자 수'].diff()

In [9]:
# na 값을 0으로 대체
df = df.fillna(0)

# float -> int
df = df.astype({'일일 확진자 수': int})

,시도명,날짜,확진자 수,지역발생 수,일일 확진자 수
0,강원,2020년 01월 20일 00시,0,0,0
1,강원,2020년 01월 21일 00시,0,0,0
2,강원,2020년 01월 22일 00시,0,0,0
3,강원,2020년 01월 23일 00시,0,0,0
4,강원,2020년 01월 24일 00시,0,0,0
...,...,...,...,...,...
675,강원,2021년 11월 26일 00시,8344,53,53
676,강원,2021년 11월 27일 00시,8437,93,93
677,강원,2021년 11월 28일 00시,8500,61,63
678,강원,2021년 11월 29일 00시,8556,55,56


In [10]:
# 특정문자 기준으로 문자열 나누고 새로운 열 추가
df['년'] = df.날짜.str.split(' ').str[0]
df['월'] = df.날짜.str.split(' ').str[1]

In [19]:
# 날짜를 지우고 년 월만 맨 앞으로
new_df = df[['년', '월', '시도명','확진자 수', '지역발생 수', '일일 확진자 수']]
# 일일 확진자 수 -> 월별 확진자 수
new_df = new_df.rename(columns={'일일 확진자 수':'월별 확진자 수'})

In [20]:
# 년, 월 기준으로 sum()
result_df = new_df.groupby(['년', '월', '시도명'], as_index=False).sum()
result_df

,년,월,시도명,확진자 수,지역발생 수,월별 확진자 수
0,2020년,01월,강원,0,0,0
1,2020년,02월,강원,43,7,7
2,2020년,03월,강원,868,22,29
3,2020년,04월,강원,1484,10,17
4,2020년,05월,강원,1692,2,4
5,2020년,06월,강원,1806,8,8
6,2020년,07월,강원,2156,4,9
7,2020년,08월,강원,3318,111,115
8,2020년,09월,강원,6426,34,35
9,2020년,10월,강원,7391,51,55
